In [55]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/LearnAboutTfrecord/First/kaggle.json ~/.kaggle/

In [3]:
!chmod 600 ~/.kaggle/kaggle.json

In [4]:

# Replace with your desired dataset
!kaggle datasets download -d heptapod/titanic

Dataset URL: https://www.kaggle.com/datasets/heptapod/titanic
License(s): DbCL-1.0
  0% 0.00/10.8k [00:00<?, ?B/s]
100% 10.8k/10.8k [00:00<00:00, 23.0MB/s]


In [14]:
!kaggle datasets download -d azeembootwala/titanic


Dataset URL: https://www.kaggle.com/datasets/azeembootwala/titanic
License(s): DbCL-1.0
... resuming from 11090 bytes (1316 bytes left) ...
 89% 10.8k/12.1k [00:00<?, ?B/s]
100% 12.1k/12.1k [00:00<00:00, 3.39MB/s]


In [22]:
# prompt: !unzip titanic.zip --y

!unzip /content/titanic.zip



Archive:  /content/titanic.zip
test_data.csv:  mismatching "local" filename (train_and_test2.csv),
         continuing with "central" filename version
replace test_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
file #2:  bad zipfile offset (local header sig):  1847


In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os


In [8]:
data=pd.read_csv('/content/train_and_test2.csv')
data.head()

,Passengerid,Age,Fare,Sex,sibsp,zero,zero.1,zero.2,zero.3,zero.4,...,zero.12,zero.13,zero.14,Pclass,zero.15,zero.16,Embarked,zero.17,zero.18,2urvived
0,1,22.0,7.2500,0,1,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,0
1,2,38.0,71.2833,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0.0,0,0,1
2,3,26.0,7.9250,1,0,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,1
3,4,35.0,53.1000,1,1,0,0,0,0,0,...,0,0,0,1,0,0,2.0,0,0,1
4,5,35.0,8.0500,0,0,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,0


In [9]:
data.isnull().sum()

,0
Passengerid,0
Age,0
Fare,0
Sex,0
sibsp,0
zero,0
zero.1,0
zero.2,0
zero.3,0
zero.4,0


In [10]:
data.dtypes

,0
Passengerid,int64
Age,float64
Fare,float64
Sex,int64
sibsp,int64
zero,int64
zero.1,int64
zero.2,int64
zero.3,int64
zero.4,int64


In [40]:
def _int64_feature(value):
  if type(value) == list:
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))
  else:
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# def _float_feature(value):
#   if type(value) == list:
#     return tf.train.Feature(float_list=tf.train.FloatList(value=value))
#   else:
#     return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _float_feature(value):
    if isinstance(value, list):
        return tf.train.Feature(float_list=tf.train.FloatList(value=[float(v) for v in value]))
    else:
        return tf.train.Feature(float_list=tf.train.FloatList(value=[float(value)]))



def _bytes_feature(value):
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  if isinstance(value, list):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))
  elif isinstance(value, bytes):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
  else:
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode('utf-8')]))

In [41]:
data.dtypes.unique()

array([dtype('int64'), dtype('float64')], dtype=object)

In [42]:
types={}
for i,j in enumerate(data.dtypes):
  if j=='object':
    types[data.columns[i]]='str'
  elif j=='int64':
    types[data.columns[i]]='int'
  elif j=='float64':
    types[data.columns[i]]='float'
  else:
    types[data.columns[i]]=str(j)
types

{'Passengerid': 'int',
 'Age': 'float',
 'Fare': 'float',
 'Sex': 'int',
 'sibsp': 'int',
 'zero': 'int',
 'zero.1': 'int',
 'zero.2': 'int',
 'zero.3': 'int',
 'zero.4': 'int',
 'zero.5': 'int',
 'zero.6': 'int',
 'Parch': 'int',
 'zero.7': 'int',
 'zero.8': 'int',
 'zero.9': 'int',
 'zero.10': 'int',
 'zero.11': 'int',
 'zero.12': 'int',
 'zero.13': 'int',
 'zero.14': 'int',
 'Pclass': 'int',
 'zero.15': 'int',
 'zero.16': 'int',
 'Embarked': 'float',
 'zero.17': 'int',
 'zero.18': 'int',
 '2urvived': 'int'}

In [43]:
values_list = list(types.values())
keys_list = list(types.keys())

In [44]:
def convert_toexample(row, column_types_list, column_names_list):
  feature_dict = {}
  for i in range(0,len(row)):
    if column_types_list[i] == 'str' or column_types_list[i] == 'byte':
      feature_dict[column_names_list[i]] = _bytes_feature(str.encode(str(row[i]))) # Encode to bytes
    elif column_types_list[i] == 'int':
      feature_dict[column_names_list[i]] = _int64_feature(int(row[i])) # Convert to int
    elif column_types_list[i] == 'float':
      feature_dict[column_names_list[i]] = _float_feature(float(row[i])) # Convert to float
  example = tf.train.Example(features=tf.train.Features(feature = feature_dict))
  return example

In [45]:
with tf.io.TFRecordWriter(path="titanic.tfrecords") as writer:
  for _, row in data.iterrows():
    example = convert_toexample(row=row, column_types_list=values_list, column_names_list=keys_list)
    writer.write(example.SerializeToString())

<ipython-input-44-05007174a79b>:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  feature_dict[column_names_list[i]] = _int64_feature(int(row[i])) # Convert to int
<ipython-input-44-05007174a79b>:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  feature_dict[column_names_list[i]] = _float_feature(float(row[i])) # Convert to float


In [46]:
import tensorflow as tf

# Define compression options
options = tf.io.TFRecordOptions(compression_type="GZIP")

# Write the TFRecord file with compression
with tf.io.TFRecordWriter(path="titanic.tfrecords.gz", options=options) as writer:
    for _, row in data.iterrows():
        example = convert_toexample(row=row, column_types_list=values_list, column_names_list=keys_list)
        writer.write(example.SerializeToString())


<ipython-input-44-05007174a79b>:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  feature_dict[column_names_list[i]] = _int64_feature(int(row[i])) # Convert to int
<ipython-input-44-05007174a79b>:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  feature_dict[column_names_list[i]] = _float_feature(float(row[i])) # Convert to float


In [47]:
raw_dataset = tf.data.TFRecordDataset("titanic.tfrecords.gz", compression_type="GZIP")


In [48]:
raw_dataset

<TFRecordDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [49]:


def get_feature_description(types):
  feature_description = {}
  for key, value in types.items():
    if value == 'str' or value == 'bytes':
      feature_description[key] = tf.io.FixedLenFeature([], tf.string)
    elif value == 'int':
      feature_description[key] = tf.io.FixedLenFeature([], tf.int64)
    elif value == 'float':
      feature_description[key] = tf.io.FixedLenFeature([], tf.float32)
  return feature_description
feature_description = get_feature_description(types)

In [50]:
def parse_example(serialized_example):
    # Parse the serialized data using the feature description
    return tf.io.parse_single_example(serialized_example, feature_description)

# Read TFRecord file
raw_dataset = tf.data.TFRecordDataset("titanic.tfrecords.gz", compression_type="GZIP")

# Parse the dataset
parsed_dataset = raw_dataset.map(parse_example)

# Print the parsed records
for parsed_record in parsed_dataset.take(5):
    print(parsed_record)


{'2urvived': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'Age': <tf.Tensor: shape=(), dtype=float32, numpy=22.0>, 'Embarked': <tf.Tensor: shape=(), dtype=float32, numpy=2.0>, 'Fare': <tf.Tensor: shape=(), dtype=float32, numpy=7.25>, 'Parch': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'Passengerid': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'Pclass': <tf.Tensor: shape=(), dtype=int64, numpy=3>, 'Sex': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'sibsp': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'zero': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'zero.1': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'zero.10': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'zero.11': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'zero.12': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'zero.13': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'zero.14': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'zero.15': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'zero.16': <tf.Tensor: shape=(), dtype=int64, n

In [54]:
parsed_dataset

<_MapDataset element_spec={'2urvived': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Age': TensorSpec(shape=(), dtype=tf.float32, name=None), 'Embarked': TensorSpec(shape=(), dtype=tf.float32, name=None), 'Fare': TensorSpec(shape=(), dtype=tf.float32, name=None), 'Parch': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Passengerid': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Pclass': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Sex': TensorSpec(shape=(), dtype=tf.int64, name=None), 'sibsp': TensorSpec(shape=(), dtype=tf.int64, name=None), 'zero': TensorSpec(shape=(), dtype=tf.int64, name=None), 'zero.1': TensorSpec(shape=(), dtype=tf.int64, name=None), 'zero.10': TensorSpec(shape=(), dtype=tf.int64, name=None), 'zero.11': TensorSpec(shape=(), dtype=tf.int64, name=None), 'zero.12': TensorSpec(shape=(), dtype=tf.int64, name=None), 'zero.13': TensorSpec(shape=(), dtype=tf.int64, name=None), 'zero.14': TensorSpec(shape=(), dtype=tf.int64, name=None), 'zero.15': TensorSpec

More compression

In [51]:
def _float_feature(value):
    if isinstance(value, list):
        return tf.train.Feature(float_list=tf.train.FloatList(value=[float(v) for v in value]))
    else:
        return tf.train.Feature(float_list=tf.train.FloatList(value=[float(value)]))


In [52]:
with tf.io.TFRecordWriter(path="titanic1.tfrecords") as writer:
  for _, row in data.iterrows():
    example = convert_toexample(row=row, column_types_list=values_list, column_names_list=keys_list)
    writer.write(example.SerializeToString())

<ipython-input-44-05007174a79b>:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  feature_dict[column_names_list[i]] = _int64_feature(int(row[i])) # Convert to int
<ipython-input-44-05007174a79b>:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  feature_dict[column_names_list[i]] = _float_feature(float(row[i])) # Convert to float


In [53]:
import tensorflow as tf

# Define compression options
options = tf.io.TFRecordOptions(compression_type="GZIP")

# Write the TFRecord file with compression
with tf.io.TFRecordWriter(path="titanic1.tfrecords.gz", options=options) as writer:
    for _, row in data.iterrows():
        example = convert_toexample(row=row, column_types_list=values_list, column_names_list=keys_list)
        writer.write(example.SerializeToString())


<ipython-input-44-05007174a79b>:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  feature_dict[column_names_list[i]] = _int64_feature(int(row[i])) # Convert to int
<ipython-input-44-05007174a79b>:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  feature_dict[column_names_list[i]] = _float_feature(float(row[i])) # Convert to float
